In [47]:
from anomstack.config import get_specs
from dashboard.data import get_data

In [48]:
metric_batch = "eirgrid"
metrics_dir = "../metrics"

In [49]:
specs = get_specs(metrics_dir)
spec = specs[metric_batch]
spec

{'db': 'duckdb',
 'table_key': 'metrics',
 'disable_dashboard': False,
 'model_path': 'local://./models',
 'model_configs': [{'model_name': 'PCA',
   'model_tag': 'pca_default',
   'model_params': {'contamination': 0.01}},
  {'model_name': 'KNN',
   'model_tag': 'knn_default',
   'model_params': {'contamination': 0.01}}],
 'model_combination_method': 'mean',
 'train_max_n': 2500,
 'train_metric_timestamp_max_days_ago': 30,
 'train_min_n': 14,
 'score_max_n': 25,
 'score_metric_timestamp_max_days_ago': 7,
 'preprocess_params': {'diff_n': 1,
  'smooth_n': 3,
  'lags_n': 5,
  'freq': None,
  'freq_agg': 'mean'},
 'alert_methods': 'email,slack',
 'alert_always': False,
 'alert_metric_timestamp_max_days_ago': 45,
 'alert_recent_n': 1,
 'alert_max_n': 250,
 'alert_smooth_n': 3,
 'alert_snooze_n': 3,
 'alert_threshold': 0.8,
 'change_metric_timestamp_max_days_ago': 45,
 'change_max_n': 250,
 'change_smooth_n': 1,
 'change_snooze_n': 3,
 'change_threshold': 3.5,
 'change_detect_last_n': 1,
 'l

In [50]:
df = get_data(spec)
df.head()

,metric_timestamp,metric_batch,metric_name,metric_value,metric_score,metric_alert,metric_llmalert,metric_change,metadata
0,2025-03-08 06:15:00,eirgrid,eirgrid_gen_exp,4303.00,NaN,NaN,NaN,NaN,[None]
1,2025-03-08 06:15:00,eirgrid,eirgrid_sys_frequency,50.02,NaN,NaN,1.0,NaN,[None]
2,2025-03-08 06:15:00,eirgrid,eirgrid_system_demand,4025.00,NaN,NaN,1.0,NaN,[None]
3,2025-03-08 06:15:00,eirgrid,eirgrid_wind_actual,2229.00,NaN,NaN,1.0,NaN,[None]
4,2025-03-09 06:15:00,eirgrid,eirgrid_gen_exp,3414.00,NaN,NaN,NaN,NaN,[None]


In [51]:
# df = df.query("metric_llmalert >= 1")


In [52]:
# extract anomaly_explanation from metadata
# metadata is the result of an array_agg of a json string
from anomstack.df.wrangle import extract_metadata

df = extract_metadata(df, "anomaly_explanation")
df

,metric_timestamp,metric_batch,metric_name,metric_value,metric_score,metric_alert,metric_llmalert,metric_change,metadata,anomaly_explanation
0,2025-03-08 06:15:00,eirgrid,eirgrid_gen_exp,4303.00,NaN,NaN,NaN,NaN,[None],None
1,2025-03-08 06:15:00,eirgrid,eirgrid_sys_frequency,50.02,NaN,NaN,1.0,NaN,[None],None
2,2025-03-08 06:15:00,eirgrid,eirgrid_system_demand,4025.00,NaN,NaN,1.0,NaN,[None],None
3,2025-03-08 06:15:00,eirgrid,eirgrid_wind_actual,2229.00,NaN,NaN,1.0,NaN,[None],None
4,2025-03-09 06:15:00,eirgrid,eirgrid_gen_exp,3414.00,NaN,NaN,NaN,NaN,[None],None
...,...,...,...,...,...,...,...,...,...,...
71,2025-03-28 06:15:00,eirgrid,eirgrid_wind_actual,3294.00,NaN,NaN,3294.0,NaN,"[None, {""anomaly_explanation"": ""Extremely high...",Extremely high value indicating a possible ano...
72,2025-03-29 06:15:00,eirgrid,eirgrid_gen_exp,3529.00,NaN,NaN,NaN,NaN,[],None
73,2025-03-29 06:15:00,eirgrid,eirgrid_sys_frequency,49.97,NaN,NaN,NaN,NaN,[],None
74,2025-03-29 06:15:00,eirgrid,eirgrid_system_demand,3973.00,NaN,NaN,NaN,NaN,[],None
